In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('ggplot')
%matplotlib inline
from tqdm import tqdm
from datasets import load_dataset

import logging
import spacy
import re


In [2]:
import pandas as pd
from src.PrepareSentenceContext import PrepareSentenceContext
from src.SentenceParser import SentenceParser
from spacy.lang.en import English

In [6]:
input_file = "Data/ChatGPT/mix/Corgi mix.txt"

with open(input_file, 'rt') as f:
    text = f.read()

full_parser = PrepareSentenceContext(sentence_parser="regex")
pout = full_parser(text)

In [7]:
pout['tag']

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

text
length
context
tag
number_in_par


In [86]:
for s in nlp(text).sents:
    for ss in s.text.split('\n'):
        print("=============")
        print(ss)

July 18 2023.
Welsh Corgi.

Introduction.

The Welsh Corgi, often simply referred to as Corgi, is a charming and beloved breed of herding dog with roots deeply entwined in the landscapes and folklore of Wales.

Renowned for their distinctive appearance, characterized by their short legs and elongated bodies, Welsh Corgis have captured the hearts of dog enthusiasts worldwide.

<edit>
Originally bred to herd cattle, sheep, and horses, Corgis are celebrated for their intelligence, agility, unwavering loyalty, and eagerness to please.
</edit>

This article delves into the fascinating history, characteristics, and unique attributes of the Welsh Corgi breed.

From their origin as indispensable herding aides in the verdant fields of Wales to their prominence as cherished family pets and even their occasional foray into pop culture, we explore the enduring appeal of these enchanting canine companions.

Moreover, we examine the different varieties of Welsh Corgis, their temperament, care requir

In [78]:
def sentence_segmenter(doc):
    return doc.split('\n')

@English.component("sentence_segmenter")
def set_sent_starts(doc):
    for sent_start_char, sent_end_char in sentence_segmenter(doc.text):
        sent = doc.char_span(sent_start_char, sent_end_char)
        sent[0].sent_start = True
        for token in sent[1:]:
            token.sent_start = False
    return doc

nlp.add_pipe("sentence_segmenter", name='sentence_segmenter', before='parser')
# for d in nlp.pipe([text], disable=["tok2vec", "tagger", "attribute_ruler", "lemmatizer"]):
#     for s in d.sents:
#         print(s)
#         print("=============")


ValueError: [E007] 'sentence_segmenter' already exists in pipeline. Existing names: ['tok2vec', 'tagger', 'sentence_segmenter', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner']

In [79]:
for s in nlp(text).sents:
    print("=============")
    print(s)


ValueError: too many values to unpack (expected 2)

In [4]:
parser = PrepareSentenceContext(engine='spacy')

In [5]:
df = pd.DataFrame(parser.parse_sentences(text))

In [8]:
df.head(10)

,text,length,context,tag,number_in_par
0,July 18 2023.,4,None,None,1
1,Welsh Corgi.\n\nIntroduction.\n,7,None,None,2
2,"The Welsh Corgi, often simply referred to as C...",33,None,None,3
3,"Renowned for their distinctive appearance, cha...",27,None,None,4
4,"Originally bred to herd cattle, sheep, and hor...",28,None,<edit>,5
5,This article delves into the fascinating histo...,20,None,None,6
6,From their origin as indispensable herding aid...,41,None,None,7
7,"Moreover, we examine the different varieties o...",28,None,None,8
8,Join us on this journey to discover the captiv...,34,None,None,9
9,History.\n,3,None,None,10


In [6]:
df[(df['tag'] == '<edit>') & (df['length'] < 15) ]

,text,length,context,tag,number_in_par
36,"With proper care, Welsh Corgis live between 11...",13,None,<edit>,37
71,She also bred several litters from her beloved...,12,None,<edit>,72
75,Their enthusiastic nature and fox-like feature...,13,None,<edit>,76


In [33]:
for token in parsed:
    if str(token) == "<EDIT>":
        token.is_sent_start = True
        token.is_sent_end = False

    if str(token) == "</EDIT>":
        token.is_sent_start = False
        token.is_sent_end = True

[(t, t.is_sent_start) for t in parsed[70:]]

[(1922, False),
 (., False),
 (, False),
 (<, True),
 (edit, False),
 (>, False),
 (He, True),
 (first, False),
 (began, False),
 (work, False),
 (in, False),
 (the, False),
 (Cubist, False),
 (style, False),
 (initially, False),
 (popularized, False),
 (by, False),
 (Picasso, False),
 (., False),
 (<, True),
 (/edit, False),
 (>, False),
 (, False),
 (Later, False),
 (,, False),
 (he, False),
 (developed, False),
 (a, False),
 (reputation, False),
 (for, False),
 (his, False),
 (avant, False),
 (-, False),
 (garde, False),
 (ideas, False),
 (and, False),
 (eccentric, False),
 (behavior, False),
 (., False),
 (, False),
 (He, True),
 (soon, False),
 (became, False),
 (associated, False),
 (with, False),
 (the, False),
 (Surrealist, False),
 (movement, False),
 (,, False),
 (which, False),
 (sought, False),
 (to, False),
 (explore, False),
 (the, False),
 (subconscious, False),
 (mind, False),
 (and, False),
 (its, False),
 (influence, False),
 (on, False),
 (creativity, False),
 (., Fa

In [20]:

import re
sents = nlp(text).sents
for s in sents:
    print("===============================================")
    a = [(t, t. is_sent_start) for t in s]
    print(s)
    tag_text = re.findall(r"(</?[a-zA-Z0-9 ]+>)", str(s))
    print(tag_text)

Salvador Dali -- Introduction.

[]
Salvador Dali was a Spanish surrealist painter born on May 11, 1904, in Figueres, Catalonia, Spain.

[]
He was one of the most famous artists of the 20th century and is best known for his surrealist paintings, which often featured dreamlike imagery, distorted figures, and symbolism.

[]
Dali began his artistic education in Madrid in 1922.

[]
<edit>
['<edit>']
He first began work in the Cubist style initially popularized by Picasso.
[]
</edit>
Later, he developed a reputation for his avant-garde ideas and eccentric behavior.

['</edit>']
He soon became associated with the Surrealist movement, which sought to explore the subconscious mind and its influence on creativity.

[]
Dali's work often featured melting clocks, barren landscapes, and grotesque figures, all of which were meant to symbolize the unconscious mind.

[]
<edit>
['<edit>']
He also repeatedly used egg imagery to symbolize birth love, and hope.
[]
</edit>
Some of his most famous works incl

In [41]:
sent

[None,
 None,
 None,
 None,
 None,
 None,
 '<edit>',
 None,
 None,
 '<edit>',
 None,
 None,
 None,
 None,
 None]

In [15]:
parser = PrepareSentenceContext(engine='spacy', context_policy='previous_sentence')
chunks = parser(text)

</edit>
 <edit> 19 August 2009: </edit>
Tottenham Hotspur host Wigan Athletic at White Hart Lane in their first game of the 2009–10 Premier League season.



</edit>
<edit>


In [145]:
null_data_file = "results/gpt2_no_context_wiki_machine.csv"
df_null = pd.read_csv(null_data_file)

from test_text_detect import get_pval_func_dict
pval_functions = get_pval_func_dict(df_null)


In [154]:
pval_functions[5](7)

array(0.04352532)

In [172]:
x = 5.403795
print(np.mean(df_null[df_null.length == 9].response > x))

0.10521849809079338


In [137]:
[chunks['text'][i] for i in range(len(chunks['text'])) if chunks['tags'][i] is not None]

['The Civil War is one of the most extensively studied and written about episodes in U.S. history.',
 'The central cause of the war was the dispute over whether slavery would be permitted to expand into the western territories, leading to more slave states, or be prevented from doing so, which was widely believed would place slavery on a course of ultimate extinction.',
 "An initial seven southern slave states responded to Lincoln's victory by seceding from the United States and, in February 1861, forming the Confederacy."]

In [173]:
255672 - 5000 - 13000 - 10000

227672

In [174]:
227672 / 2

113836.0